In [ ]:
!pip install transformers pandas openpyxl ipywidgets

In [ ]:
import io
import re
import pandas as pd
from google.colab import files
import ipywidgets as widgets
from IPython.display import display
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline


In [ ]:
print("Upload file Excel mapping branch (opsional) [misal: Branchcode Training data.xlsx]:")
uploaded_branch = files.upload()
if len(uploaded_branch) > 0:
    branch_filename = list(uploaded_branch.keys())[0]
    branch_df = pd.read_excel(branch_filename)
    branch_df["KODE CAB"] = branch_df["KODE CAB"].astype(str).str.strip()
    branch_df["brand_name"] = branch_df["brand_name"].astype(str).str.strip()
    branch_mapping = dict(zip(branch_df["KODE CAB"], branch_df["brand_name"]))
    print("Branch Mapping:", branch_mapping)
else:
    branch_mapping = {}
    print("Tidak ada file branch mapping diupload.")


Upload file Excel mapping branch (opsional) [misal: Branchcode Training data.xlsx]:


Saving Branchcode Training data.xlsx to Branchcode Training data (4).xlsx
Branch Mapping: {'10100': 'JAKARTA PUSAT 2', '10200': 'CILEGON', '10300': 'TANGERANG', '10400': 'BOGOR', '10500': 'DEPOK', '10600': 'JAKARTA BARAT', '10700': 'SUKABUMI', '10800': 'BEKASI', '10900': 'KARAWANG', '11000': 'CIKARANG', '11100': 'CILEDUG', '11200': 'TAMAN PALEM', '11300': 'JATI UWUNG', '11600': 'PASAR MINGGU', '11700': 'BUNGUR', '11900': 'SERANG', '12500': 'CIKUPA', '12600': 'CILEUNGSI', '12800': 'JAKARTA UTARA', '12900': 'CIKAMPEK', '13000': 'CIBINONG', '13200': 'SERANG II', '13300': 'DEPOK II', '13500': 'PAMULANG', '13600': 'KABUPATEN BOGOR', '13700': 'PONDOK GEDE', '14100': 'RANGKASBITUNG', '14200': 'BEKASI II', '14300': 'PANDEGLANG', '14500': 'DEPOK III', '14600': 'CIANJUR', '14800': 'SUKABUMI II', '15500': 'TANGERANG V', '15600': 'JAKARTA UTARA II', '15700': 'BOGOR II', '15800': 'JAKARTA', '15900': 'BEKASI III', '16000': 'KARAWANG II', '16200': 'JAKARTA SELATAN II', '16300': 'BOGOR IV', '16400': '

In [ ]:
print("Upload file TXT data transaksi (misal: 18022025 real.txt):")
uploaded_txt = files.upload()
if len(uploaded_txt) > 0:
    txt_filename = list(uploaded_txt.keys())[0]
    print("Uploaded TXT file:", txt_filename)
else:
    txt_filename = None
    print("Tidak ada file TXT yang diupload.")


Upload file TXT data transaksi (misal: 18022025 real.txt):


Saving 18022025 real.txt to 18022025 real (4).txt
Uploaded TXT file: 18022025 real (4).txt


In [ ]:
print("Upload file DATA TRANSAKSI BNI (misal: DATA TRANSAKSI BNI.xlsx), gunakan sheet 'BNI180225':")
uploaded_bni = files.upload()
if len(uploaded_bni) > 0:
    bni_filename = list(uploaded_bni.keys())[0]
    df_bni = pd.read_excel(bni_filename, sheet_name="BNI180225")
    print("Kolom pada file BNI:", df_bni.columns.tolist())

    # Ubah nama kolom nilai nominal
    if "AP_INVOICE_AMOUNT" in df_bni.columns:
        df_bni.rename(columns={"AP_INVOICE_AMOUNT": "nominal"}, inplace=True)
    elif "AMOUNT" in df_bni.columns:
        df_bni.rename(columns={"AMOUNT": "nominal"}, inplace=True)
    else:
        print("Kolom nilai nominal tidak ditemukan. Pastikan file memiliki kolom AMOUNT atau AP_INVOICE_AMOUNT.")

    # Ubah nama kolom branch id
    if "AP_BRANCH_ID" not in df_bni.columns:
        if "BRANCH_ID" in df_bni.columns:
            df_bni.rename(columns={"BRANCH_ID": "AP_BRANCH_ID"}, inplace=True)
        else:
            print("Kolom branch id tidak ditemukan. Pastikan file memiliki kolom BRANCH_ID atau AP_BRANCH_ID.")

    # Konversi kolom nominal ke numeric
    if "nominal" in df_bni.columns:
        df_bni["nominal"] = pd.to_numeric(df_bni["nominal"], errors="coerce")

    print("File DATA TRANSAKSI BNI berhasil diupload (sheet BNI180225).")
else:
    df_bni = pd.DataFrame()
    print("Tidak ada file DATA TRANSAKSI BNI diupload.")


Upload file DATA TRANSAKSI BNI (misal: DATA TRANSAKSI BNI.xlsx), gunakan sheet 'BNI180225':


Saving DATA TRANSAKSI BNI.xlsx to DATA TRANSAKSI BNI (4).xlsx
Kolom pada file BNI: ['AP_BRANCH_ID', 'AP_INVOICE_NO', 'AP_PV_NO', 'AP_SUPPLIER_BANK', 'AP_SUPPLIER_BANK_ACCT', "TO_CHAR(A.AP_INV_PAID_DATE,'DD-MM-YYYY')", 'AP_INVOICE_AMOUNT', 'COY_ID', 'AP_CREATED_BY']
File DATA TRANSAKSI BNI berhasil diupload (sheet BNI180225).


In [ ]:
print("Memuat model NER...")
model_name = "cahya/bert-base-indonesian-ner"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)
nlp = pipeline("ner", model=model, tokenizer=tokenizer)
print("Model NER berhasil dimuat.")


Memuat model NER...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the model checkpoint at cahya/bert-base-indonesian-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Bert

Model NER berhasil dimuat.


In [ ]:
def parse_line(line):
    """
    Memecah baris berdasarkan '|' dengan aturan:
      - Kolom terakhir = Balance
      - Kolom kedua terakhir = DB/CR
      - Kolom ketiga terakhir = Amount
      - Kolom 0 hingga 3: No, PostDate, Branch, JournalNo
      - Sisanya (index 4 sampai -3) digabung sebagai Description.
    """
    parts = line.split("|")
    if len(parts) < 8:
        return None
    balance = parts[-1].strip()
    dbcr = parts[-2].strip()
    amount = parts[-3].strip()
    no_ = parts[0].strip()
    post_date = parts[1].strip()
    branch = parts[2].strip()
    journal_no = parts[3].strip()
    description = "|".join(parts[4:-3]).strip()
    return {
        "No": no_,
        "PostDate": post_date,
        "Branch": branch,
        "JournalNo": journal_no,
        "Description": description,
        "Amount": amount,
        "DBCR": dbcr,
        "Balance": balance
    }

def extract_branch_code(text):
    """
    Mencoba mengekstrak kode cabang dari teks:
    1) Coba cari pola VA 16 digit (98822222XXX00000) → ambil XXX.
    2) Jika tidak ditemukan, cari 3 digit (word boundary) di teks.
    3) Jika tidak ada, return "000".
    """
    # Step 1: Regex untuk 16 digit
    match_16 = re.search(r"(98822222\d{3}00000)", text)
    if match_16:
        return match_16.group(1)[8:11]
    # Step 2: Cari 3 digit
    match_3 = re.search(r"\b(\d{3})\b", text)
    if match_3:
        return match_3.group(1)
    return "000"

def summarize_description_by_segments(cleaned_desc, num_segments=2):
    """
    Merangkum deskripsi dengan mengambil 'num_segments' segmen terakhir
    dari deskripsi, dipisahkan dengan delimiter '|'.
    """
    segments = cleaned_desc.split("|")
    segments = [seg.strip() for seg in segments if seg.strip() != ""]
    if not segments:
        return ""
    if len(segments) <= num_segments:
        return " | ".join(segments)
    else:
        return " | ".join(segments[-num_segments:])


In [ ]:
data_lines = []
header_found = False

if txt_filename is not None:
    with io.open(txt_filename, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not header_found:
                if line.startswith("No.|Post Date|Branch"):
                    header_found = True
                continue
            if line and line[0].isdigit():
                row_dict = parse_line(line)
                if row_dict is not None:
                    if row_dict["DBCR"].strip().upper() != "C":
                        continue
                    desc = row_dict["Description"]
                    amt_str = row_dict["Amount"]

                    branch_code = extract_branch_code(desc)

                    try:
                        amt_val = float(amt_str)
                    except:
                        amt_val = None

                    summary_desc = summarize_description_by_segments(desc, num_segments=2)

                    data_lines.append({
                        "kode cabang": branch_code,
                        "keterangan": summary_desc,
                        "nominal": amt_val
                    })
    df_main = pd.DataFrame(data_lines)
    print("Processed Output dari TXT berhasil dibuat.")
    print(df_main.head(10))
else:
    df_main = pd.DataFrame()
    print("Tidak ada file TXT yang diproses.")


Processed Output dari TXT berhasil dibuat.
  kode cabang                                         keterangan     nominal
0         418  PEMINDAHAN DARI 4714717778 Sdr MOH  RIYAJI | 9...  26341500.0
1         613  PEMINDAHAN DARI 407531430 Ibu SILVANA  SAPRIL ...   6472000.0
2         221  PEMINDAHAN DARI 1111509720 Bpk   NOFRIANTO | 9...   1300000.0
3         501  FEDERAL INTERNATIONAL FINANCE, | 9882222250100...  79355000.0
4         000                    SETOR TUNAI | Sdr DEDE SURYAMAN    724000.0
5         000    PEMINDAHAN DARI 1888488680 Sdr   RIO | admin 20   3214000.0
6         000    PEMINDAHAN DARI 1888488680 Sdr   RIO | admin 10   2851000.0
7         000  TRANSFER DARI | PEMINDAHAN DARI 768925263 Sdri...   3360000.0
8         708  Sdri DITA SILA NORMAWATI | 9882222270800000 FI...   9510000.0
9         708  Sdri DITA SILA NORMAWATI | 9882222270800000 FI...  20000000.0


In [ ]:
# Buat key di df_main
if not df_main.empty:
    df_main["kode cabang"] = df_main["kode cabang"].astype(str).str.strip()
    # Konversi nominal ke integer untuk key (asumsi tidak ada desimal penting)
    df_main["nominal_int"] = df_main["nominal"].round(0).astype(int)
    df_main["match_key"] = df_main["kode cabang"] + "_" + df_main["nominal_int"].astype(str)
    print("Contoh key df_main:")
    print(df_main[["kode cabang", "nominal_int", "match_key"]].head())
else:
    print("df_main kosong.")

# Buat key di df_bni
if not df_bni.empty:
    df_bni["AP_BRANCH_ID"] = df_bni["AP_BRANCH_ID"].astype(str).str.strip()
    df_bni["short_branch"] = df_bni["AP_BRANCH_ID"].str[:3]
    df_bni["nominal_int"] = df_bni["nominal"].round(0).astype(int)
    df_bni["match_key"] = df_bni["short_branch"] + "_" + df_bni["nominal_int"].astype(str)
    print("\nContoh key df_bni:")
    print(df_bni[["AP_BRANCH_ID", "short_branch", "nominal_int", "match_key"]].head())
else:
    print("df_bni kosong.")

# Lakukan merge berdasarkan match_key
if not df_main.empty:
    df_merge = pd.merge(df_main, df_bni[["match_key"]], on="match_key", how="left", indicator=True)
    df_merge["status"] = df_merge["_merge"].apply(lambda x: "match" if x=="both" else "unmatch")
    df_match = df_merge[["kode cabang", "nominal", "status"]].copy()
    print("\nContoh hasil matching:")
    print(df_match.head(10))
else:
    df_match = pd.DataFrame()
    print("Tidak ada data untuk matching.")


Contoh key df_main:
  kode cabang  nominal_int     match_key
0         418     26341500  418_26341500
1         613      6472000   613_6472000
2         221      1300000   221_1300000
3         501     79355000  501_79355000
4         000       724000    000_724000

Contoh key df_bni:
  AP_BRANCH_ID short_branch  nominal_int     match_key
0        61300          613      6472000   613_6472000
1        41800          418     26341500  418_26341500
2        61200          612     10000000  612_10000000
3        20700          207      6275000   207_6275000
4        20800          208      1000000   208_1000000

Contoh hasil matching:
  kode cabang     nominal   status
0         418  26341500.0    match
1         613   6472000.0    match
2         221   1300000.0    match
3         501  79355000.0  unmatch
4         000    724000.0  unmatch
5         000   3214000.0  unmatch
6         000   2851000.0  unmatch
7         000   3360000.0  unmatch
8         708   9510000.0    match
9         

In [ ]:
output_file = "final_output.xlsx"
with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
    df_main_no_key = df_main.drop(columns=["match_key", "nominal_int"], errors="ignore")
    df_main_no_key.to_excel(writer, sheet_name="Processed Output", index=False)

    if not df_match.empty:
        df_match.to_excel(writer, sheet_name="Matching Results", index=False)

print(f"\nProses selesai. Output disimpan di file: {output_file}")



Proses selesai. Output disimpan di file: final_output.xlsx


In [ ]:
def on_download_button_clicked(b):
    files.download(output_file)

download_button = widgets.Button(
    description="Download Final Output",
    disabled=False,
    button_style="success",
    tooltip="Klik untuk mengunduh file output",
    icon="download"
)
download_button.on_click(on_download_button_clicked)
display(download_button)


Button(button_style='success', description='Download Final Output', icon='download', style=ButtonStyle(), tool…

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>